In [1]:
# Importar librerías mágicas
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_curve, f1_score, classification_report
import optuna
import joblib
import xgboost as xgb
import os

c:\Users\Administrator\Desktop\proyecto10\nlp_grupo_5_proyecto_10\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/fintihlupik/NLP-sentimental/refs/heads/master/data/youtoxic_english_1000%20-%20youtoxic_english_1000.csv")


In [7]:
# Definir directorios
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(BASE_DIR, 'processed')  # Corregido
models_dir = os.path.join(BASE_DIR, 'models')

# Cargar datos vectorizados
X_train = joblib.load(os.path.join(data_dir, 'X_train_tfidf.pkl'))
X_test = joblib.load(os.path.join(data_dir, 'X_test_tfidf.pkl'))

# Cargar etiquetas
y_train = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))['IsToxic']
y_test = pd.read_csv(os.path.join(data_dir, 'y_test.csv'))['IsToxic']

In [9]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_curve, f1_score, classification_report

# Configuración de validación cruzada estratificada
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Modelo inicial con Naive Bayes
nb_model = MultinomialNB()

# Evaluación inicial con validación cruzada
y_pred_proba = cross_val_predict(nb_model, X_train, y_train, cv=skf, method='predict_proba')[:, 1]

# Calcular métricas de evaluación
precision, recall, thresholds = precision_recall_curve(y_train, y_pred_proba)
f1_scores = 2 * recall * precision / (recall + precision + 1e-6)

# Imprimir resultados iniciales
print(f"F1-Score inicial: {max(f1_scores):.4f}")

# Reporte de clasificación inicial
optimal_threshold = thresholds[np.argmax(f1_scores)]
y_pred = (y_pred_proba >= optimal_threshold).astype(int)
print("Reporte de clasificación inicial:")
print(classification_report(y_train, y_pred))

F1-Score inicial: 0.7077
Reporte de clasificación inicial:
              precision    recall  f1-score   support

           0       0.85      0.42      0.56       430
           1       0.58      0.92      0.71       370

    accuracy                           0.65       800
   macro avg       0.72      0.67      0.64       800
weighted avg       0.73      0.65      0.63       800



In [10]:
import optuna

# Definir función objetivo para Optuna
def objective(trial):
    # Sugerir un valor para el hiperparámetro alpha
    alpha = trial.suggest_loguniform('alpha', 1e-3, 10.0)
    
    # Crear el modelo con el valor de alpha sugerido
    model = MultinomialNB(alpha=alpha)
    
    # Validación cruzada estratificada
    y_pred_proba = cross_val_predict(model, X_train, y_train, cv=skf, method='predict_proba')[:, 1]
    
    # Calcular F1-score
    precision, recall, thresholds = precision_recall_curve(y_train, y_pred_proba)
    f1_scores = 2 * recall * precision / (recall + precision + 1e-6)
    
    # Retornar el mejor F1-score
    return max(f1_scores)

# Crear un estudio de Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Ejecutar 50 pruebas

# Obtener el mejor hiperparámetro
best_alpha = study.best_params['alpha']
print(f"Mejor valor de alpha: {best_alpha}")

[I 2025-07-07 09:24:56,192] A new study created in memory with name: no-name-f44a447c-15fb-4358-a1cd-45cbcc54a904
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16688\3703045900.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-3, 10.0)
[I 2025-07-07 09:24:56,249] Trial 0 finished with value: 0.7105258335741194 and parameters: {'alpha': 0.13717825285703245}. Best is trial 0 with value: 0.7105258335741194.
C:\Users\Administrator\AppData\Local\Temp\ipykernel_16688\3703045900.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-3, 10.0)
[I 2025-07-07 09:24:56,357] Trial 1 finish

Mejor valor de alpha: 0.5163020204210761


In [11]:
# Entrenar el modelo con el mejor valor de alpha encontrado por Optuna
best_model = MultinomialNB(alpha=best_alpha)
best_model.fit(X_train, y_train)

# Obtener las probabilidades predichas en el conjunto de entrenamiento
y_pred_proba = best_model.predict_proba(X_train)[:, 1]

# Calcular precisión, recall y F1-score para diferentes umbrales
precision, recall, thresholds = precision_recall_curve(y_train, y_pred_proba)
f1_scores = 2 * recall * precision / (recall + precision + 1e-6)

# Encontrar el umbral que maximiza el F1-score
optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f"Mejor umbral para F1-score: {optimal_threshold:.4f}")

# Evaluar el modelo con el umbral óptimo
y_pred_optimal = (y_pred_proba >= optimal_threshold).astype(int)
print("Reporte de clasificación con umbral optimizado:")
print(classification_report(y_train, y_pred_optimal))

Mejor umbral para F1-score: 0.4861
Reporte de clasificación con umbral optimizado:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       430
           1       0.96      0.98      0.97       370

    accuracy                           0.97       800
   macro avg       0.97      0.97      0.97       800
weighted avg       0.97      0.97      0.97       800



In [12]:
from sklearn.metrics import accuracy_score

# 1. Evaluación con el modelo inicial (umbral predeterminado de 0.5)
initial_model = MultinomialNB()
initial_model.fit(X_train, y_train)
y_pred_initial = initial_model.predict(X_train)
initial_f1 = f1_score(y_train, y_pred_initial)
initial_accuracy = accuracy_score(y_train, y_pred_initial)

# 2. Evaluación con el modelo optimizado (mejor alpha, umbral predeterminado de 0.5)
optimized_model = MultinomialNB(alpha=best_alpha)
optimized_model.fit(X_train, y_train)
y_pred_optimized = optimized_model.predict(X_train)
optimized_f1 = f1_score(y_train, y_pred_optimized)
optimized_accuracy = accuracy_score(y_train, y_pred_optimized)

# 3. Evaluación con el modelo optimizado y el mejor umbral
y_pred_best_threshold = (y_pred_proba >= optimal_threshold).astype(int)
best_threshold_f1 = f1_score(y_train, y_pred_best_threshold)
best_threshold_accuracy = accuracy_score(y_train, y_pred_best_threshold)

# Comparación de métricas
print("Comparación de métricas:")
print(f"{'Modelo':<30}{'F1-Score':<15}{'Accuracy':<15}")
print(f"{'Modelo inicial (umbral 0.5)':<30}{initial_f1:<15.4f}{initial_accuracy:<15.4f}")
print(f"{'Modelo optimizado (umbral 0.5)':<30}{optimized_f1:<15.4f}{optimized_accuracy:<15.4f}")
print(f"{'Modelo optimizado (mejor umbral)':<30}{best_threshold_f1:<15.4f}{best_threshold_accuracy:<15.4f}")

Comparación de métricas:
Modelo                        F1-Score       Accuracy       
Modelo inicial (umbral 0.5)   0.9392         0.9450         
Modelo optimizado (umbral 0.5)0.9609         0.9637         
Modelo optimizado (mejor umbral)0.9665         0.9688         


In [13]:
# Selección del mejor modelo según F1-score
if best_threshold_f1 >= max(initial_f1, optimized_f1):
    final_model = optimized_model
    final_threshold = optimal_threshold
    print("El modelo optimizado con el mejor umbral fue seleccionado como el mejor modelo.")
else:
    final_model = optimized_model if optimized_f1 > initial_f1 else initial_model
    final_threshold = 0.5
    print("El modelo optimizado o inicial con umbral 0.5 fue seleccionado como el mejor modelo.")

# Entrenar el modelo final con los datos de entrenamiento completos
final_model.fit(X_train, y_train)

# Evaluar el modelo final en el conjunto de prueba
y_test_proba = final_model.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_proba >= final_threshold).astype(int)

# Reporte de clasificación en el conjunto de prueba
from sklearn.metrics import classification_report
print("Reporte de clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_test_pred))

El modelo optimizado con el mejor umbral fue seleccionado como el mejor modelo.
Reporte de clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

           0       0.67      0.70      0.69       108
           1       0.63      0.60      0.61        92

    accuracy                           0.66       200
   macro avg       0.65      0.65      0.65       200
weighted avg       0.65      0.66      0.65       200



In [23]:
models_dir = os.path.join(BASE_DIR, 'models')
print(f"Ruta de models_dir: {models_dir}")

Ruta de models_dir: c:\Users\Administrator\Desktop\proyecto10\nlp_grupo_5_proyecto_10\data\models


In [25]:
import joblib
import os

# Ajustar la ruta de models_dir para que apunte fuera de 'data'
models_dir = os.path.join(BASE_DIR, 'models')  # Asegúrate de que apunta al directorio correcto

# Crear la carpeta 'models' si no existe
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"Carpeta 'models' creada en: {models_dir}")
else:
    print(f"Usando la carpeta existente: {models_dir}")

# Ruta para guardar el modelo
model_path = os.path.join(models_dir, 'naive_bayes_best_model.pkl')

# Guardar el modelo final
joblib.dump(final_model, model_path)
print(f"Modelo guardado en: {model_path}")

Carpeta 'models' creada en: c:\Users\Administrator\Desktop\proyecto10\nlp_grupo_5_proyecto_10\data\models
Modelo guardado en: c:\Users\Administrator\Desktop\proyecto10\nlp_grupo_5_proyecto_10\data\models\naive_bayes_best_model.pkl
